# Design Project 3  
*Attitude‑Control Analysis & Simulation of a 4‑Wheel Spacecraft*

**Authors:** Jason Li and Joshua Veranga  
**Course:** AE353: Aerospace Control Systems  

> *Abstract* — We design, tune, and validate an attitude‑control architecture for a rigid‑body spacecraft actuated by four reaction wheels. After deriving the full nonlinear equations of motion, we linearize about a rest‑to‑dock equilibrium and synthesize an LQR controller with a Kalman‑filter state observer. 


## 1 - Imports  

In [ ]:
import sympy as sym
import numpy as np
import ae353_spacecraft_design as design
import ae353_spacecraft_simulate as simulate
from sympy.utilities.lambdify import lambdify
from scipy.linalg import solve_continuous_are
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

## 2 - Wheel & Star Geometry  


In [ ]:
wheels = [
    {"alpha":  np.pi/2,   "delta": 0.0},        #  (0, +2.2, 0)
    {"alpha": -np.pi/2,   "delta": 0.0},        #  (0, -2.2, 0)
    {"alpha":  np.pi,     "delta": 0.0},        #  (-2.2, 0, 0)
    {"alpha":  0.0,       "delta": -np.pi/2}    #  (0, 0, -2.2)
]

m, J = design.create_spacecraft(wheels)

stars = [
    {'alpha':  0.0577, 'delta':  0.00},
    {'alpha': -0.0289, 'delta':  0.05},
    {'alpha': -0.0289, 'delta': -0.05}
]

design.create_stars(stars)

## 3 - Spacecraft Controller Setup

### 3.1 - Equations of Motion

In [ ]:
# Define yaw, pitch, roll angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Compute resultant torques
T1 = - tau_1 * sym.Matrix(wheels[0]['xyz']) / np.linalg.norm(wheels[0]['xyz'])
T2 = - tau_2 * sym.Matrix(wheels[1]['xyz']) / np.linalg.norm(wheels[1]['xyz'])
T3 = - tau_3 * sym.Matrix(wheels[2]['xyz']) / np.linalg.norm(wheels[2]['xyz'])
T4 = - tau_4 * sym.Matrix(wheels[3]['xyz']) / np.linalg.norm(wheels[3]['xyz'])
T = sym.nsimplify(T1 + T2 + T3 + T4)

# Define rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

# Define the transformation from angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Define euler's equations
Jx, Jy, Jz = [sym.nsimplify(j) for j in np.diag(J)]
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                    [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                    [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# Define equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

alpha, delta = sym.symbols('alpha, delta')

# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rz * Ry * Rx

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
r = sym.nsimplify(design.scope_radius)
p_star_in_image = (1 / r) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                        [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

### 3.2 - Solve for the Controller Gain Matrices

In [ ]:
# --- Define symbols ---
psi, theta, phi = sym.symbols('psi, theta, phi')
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Define state and input
x = sym.Matrix([psi, theta, phi, w_x, w_y, w_z])
u = sym.Matrix([tau_1, tau_2, tau_3, tau_4])


# Compute Jacobians
A = f.jacobian(x)
B = f.jacobian(u)

# Define equilibrium point (rest + zero orientation + zero torque)
eq_pt = {
    psi: 0, theta: 0, phi: 0,
    w_x: 0, w_y: 0, w_z: 0,
    tau_1: 0, tau_2: 0, tau_3: 0, tau_4: 0
}

# Substitute equilibrium into Jacobians
A_eq = A.subs(eq_pt)
B_eq = B.subs(eq_pt)

# Lambdify to get numerical versions
A_func = lambdify((), A_eq)
B_func = lambdify((), B_eq)

# Evaluate as numpy arrays
A_num = np.array(A_func(), dtype=np.float64)
B_num = np.array(B_func(), dtype=np.float64)

# Convert symbolic A_eq and B_eq to numpy arrays
A_eq_np = np.array(A_eq).astype(np.float64)
B_eq_np = np.array(B_eq).astype(np.float64)

# Define Q and R as numpy arrays
Q = np.diag([
    10000, 10000, 10000,    # phi, theta, psi → keep spacecraft pointing correctly
    1, 1, 1                 # w_x, w_y, w_z → care less about rotational speed
])

R = np.eye(4)  # penalize all 4 torque inputs equally

# Solve CARE: A.T P + P A - P B R^-1 B.T P + Q = 0
P = solve_continuous_are(A_eq_np, B_eq_np, Q, R)

# Compute LQR gain K
K = np.linalg.inv(R) @ B_eq_np.T @ P

g_all = []
for s in stars:
    g_i = g.subs({alpha: s['alpha'], delta: s['delta']})
    g_all.append(g_i)
# Stack measurement model
h = sym.Matrix.vstack(*g_all)  # size 2N x 1

# Derivative w.r.t orientation part of state
x_sensor = sym.Matrix([psi, theta, phi])
C = h.jacobian(x_sensor)

# Evaluate at equilibrium (psi=theta=phi=0)
C_eq = C.subs({psi: 0, theta: 0, phi: 0})

# Convert to numpy function
C_func = lambdify((), C_eq)
C_num = np.array(C_func(), dtype=np.float64)

# Pad with zeros to make it 2N x 6 (full state)
C_full = np.hstack((C_num, np.zeros((C_num.shape[0], 3))))

# Define observer tuning matrices 
Qo = np.eye(A_num.shape[0]) * 1     # 6x6
Ro = np.eye(C_full.shape[0]) * 1    # 2N x 2N

# Solve the dual LQR problem
Po = solve_continuous_are(A_num.T, C_full.T, Qo, Ro)

# Compute observer gain
L = Po @ C_full.T @ np.linalg.inv(Ro)

## 4 - Controller

### 4.1 - Define Controller Class

In [ ]:
class Controller:
    def __init__(self, A_eq_np, B_eq_np, C_full, K, L, dt=0.01):
        self.A = A_eq_np        # (6x6) system dynamics matrix
        self.B = B_eq_np        # (6x4) control input matrix
        self.C = C_full         # (2N x 6) measurement matrix
        self.K = K              # (4x6) LQR gain
        self.L = L              # (6 x 2N) observer gain
        self.dt = dt
        self.xhat = np.zeros(6)  # initial state estimate
        
    def reset(self):
        pass
    
    def run(self, t, star_measurements):
        """
        The variable t is the current time.

        The variable star_measurements is a 1d array of length twice the
        number N of stars:

            [y_1, z_1, y_2, z_2, ..., y_N, z_N]
        
        The image coordinates y_i and z_i of the i'th star (for i = 1, ..., N)
        are at index 2 * i - 2 and 2 * i - 1 of this array, respectively.
        """
        
        y = np.array(star_measurements)

        # Predicted measurement from current estimate
        y_hat = self.C @ self.xhat

        # Compute control
        u = -self.K @ self.xhat

        # Update observer
        dxhat = self.A @ self.xhat + self.B @ u + self.L @ (y - y_hat)
        self.xhat += dxhat * self.dt  

        return tuple(u)

### 4.2 - Instantiate controller  

In [ ]:
controller = Controller(A_eq_np, B_eq_np, C_full, K, L)

## 5 - Controller Error

### 5.1 - Initialize Simulator

In [ ]:
simulator = simulate.Simulator(
    scope_noise=0.0,
    display=False,
    seed=None,
)

### 5.2 - Reset Simulator and Controller

In [ ]:
simulator.reset(
    initial_conditions    =None,
    scope_noise           =0.01,       # <-- standard deviation of each image coordinate of each star tracker measurement
    space_debris          =True,      # <-- whether or not there is space debris
    docking_time          =30.00      # <-- how long it takes for the space-cat to dock with the spacecraft
)

controller.reset()

### 5.3 - Run Simulation

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=32.00,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
    print_debug=False,    # <-- optional (whether to print debug text - this is recommended when saving video)
)

### 5.4 - Plot Error Curves

In [ ]:
# --- Trim to min length to sync data and xhat_log ---
min_len = min(len(controller.xhat_log), len(data['psi']))
psi = np.array(data['psi'][:min_len])
theta = np.array(data['theta'][:min_len])
phi = np.array(data['phi'][:min_len])
t = np.array(data['t'][:min_len])
xhat = np.array(controller.xhat_log[:min_len])

# Compute estimation errors (xhat - x)
psi_err = xhat[:, 0] - psi
theta_err = xhat[:, 1] - theta
phi_err = xhat[:, 2] - phi

# --- Standard color palette ---
colors = {
    'psi': 'tab:blue',
    'theta': 'tab:green',
    'phi': 'tab:orange'
}

# --- Figure 1: True Euler Angles Only (x - x_eq) ---
fig1, axs1 = plt.subplots(3, 1, figsize=(22, 12), sharex=True)
fig1.subplots_adjust(right=0.78, hspace=0.3)

axs1[0].plot(t, psi, color=colors['psi'], linewidth=1)
axs1[0].set_ylabel(r'$\psi - \psi_{eq}$ (rad)', fontsize=24)
axs1[0].grid(True, linestyle='--', linewidth=0.6)
axs1[0].set_xlim([0, 60])
axs1[0].tick_params(labelsize=14)

axs1[1].plot(t, theta, color=colors['theta'], linewidth=1)
axs1[1].set_ylabel(r'$\theta - \theta_{eq}$ (rad)', fontsize=24)
axs1[1].grid(True, linestyle='--', linewidth=0.6)
axs1[1].set_xlim([0, 60])
axs1[1].tick_params(labelsize=14)

axs1[2].plot(t, phi, color=colors['phi'], linewidth=1)
axs1[2].set_xlabel("Time (s)", fontsize=24)
axs1[2].set_ylabel(r'$\phi - \phi_{eq}$ (rad)', fontsize=24)
axs1[2].grid(True, linestyle='--', linewidth=0.6)
axs1[2].set_xlim([0, 60])
axs1[2].tick_params(labelsize=14)

plt.tight_layout()
plt.show()

# --- Figure 2: Estimation Error Only (xhat - x) ---
fig2, axs2 = plt.subplots(3, 1, figsize=(22, 12), sharex=True)
fig2.subplots_adjust(right=0.78, hspace=0.3)

axs2[0].plot(t, psi_err, color=colors['psi'], linewidth=1)
axs2[0].set_ylabel(r'$\hat{\psi} - \psi$ (rad)', fontsize=24)
axs2[0].grid(True, linestyle='--', linewidth=0.6)
axs2[0].set_xlim([0, 60])
axs2[0].tick_params(labelsize=14)

axs2[1].plot(t, theta_err, color=colors['theta'], linewidth=1)
axs2[1].set_ylabel(r'$\hat{\theta} - \theta$ (rad)', fontsize=24)
axs2[1].grid(True, linestyle='--', linewidth=0.6)
axs2[1].set_xlim([0, 60])
axs2[1].tick_params(labelsize=14)

axs2[2].plot(t, phi_err, color=colors['phi'], linewidth=1)
axs2[2].set_xlabel("Time (s)", fontsize=18)
axs2[2].set_ylabel(r'$\hat{\phi} - \phi$ (rad)', fontsize=24)
axs2[2].grid(True, linestyle='--', linewidth=0.6)
axs2[2].set_xlim([0, 60])
axs2[2].tick_params(labelsize=14)

plt.tight_layout()
plt.show()

## 6 - Eta Sweep Simulation

### 6.1 - Run Simulation

In [ ]:
# Define the different Q weight values to test
q_values = [1, 10, 100, 1000, 10000, 100000]
n_trials = 100
results = []

for q_val in q_values:
    Q = np.diag([q_val, q_val, q_val, 1, 1, 1])
    P = solve_continuous_are(A_eq_np, B_eq_np, Q, R)
    K = np.linalg.inv(R) @ B_eq_np.T @ P

    # Rebuild controller with new K
    controller = Controller(A_eq_np, B_eq_np, C_full, K, L)
    
    success_count = 0
    for trial in range(n_trials):
        simulator = simulate.Simulator(display=False, seed=None)
        simulator.reset(
            initial_conditions=None,
            scope_noise=0.1,
            space_debris=True,
            docking_time=20.0,
        )
        controller.reset()
        data = simulator.run(controller, max_time=25.0, print_debug=False)

        if simulator.has_docked():
            success_count += 1
        #print(f"Q={q_val}, Trial {trial+1}/{n_trials}, Docked={simulator.has_docked()}")

    success_rate = success_count / n_trials * 100
    results.append((q_val, success_rate))
    #print(f"Q=[{q_val}, {q_val}, {q_val}, 1, 1, 1]: Success rate = {success_rate:.1f}%")

# Final results summary
print("\n--- Summary ---")
for q_val, success_rate in results:
    print(f"Q=[{q_val}, {q_val}, {q_val}, 1, 1, 1] -> Success Rate: {success_rate:.1f}%")

### 6.2 - Plot Simulation Data

In [ ]:
# Unpack Q values and success rates from results
q_vals, success_rates = zip(*results)

# Plotting
plt.figure(figsize=(10, 6))
plt.semilogx(q_vals, success_rates, marker='o', linestyle='-', linewidth=2)

#plt.title("Success Rate vs Q Matrix Weight", fontsize=14)
plt.xlabel("Q Value (log scale)", fontsize=12)
plt.ylabel("Success Rate (%)", fontsize=12)
plt.grid(True, which="both", linestyle='--', linewidth=0.7)
plt.xticks(q_vals, labels=[str(q) for q in q_vals])
plt.ylim(0, 100)
plt.tight_layout()
plt.show()